In [37]:
import numpy as np
from scipy import signal, io
import matplotlib.pyplot as plt
from acquisition_tools import cal_matrix, estimate_rpm
from tkinter import Tk
from tkinter.filedialog import askopenfilename
%matplotlib qt

plt.rcParams['text.usetex'] = False
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['font.size'] = 16

cal_mat = cal_matrix()

In [38]:
# Load the .mat file
# Open a file dialog to choose a .mat file
root = Tk()
# root.withdraw()  # Hide the main window
file_path = askopenfilename(initialdir='./results', filetypes=[("MAT files", "*.mat")])
root.destroy()  # Destroy the main window

exp_data = io.loadmat(file_path)

In [39]:
diameter_in = exp_data['D_inch']
pitch_in = exp_data['pitch']

diameter_m = diameter_in * 0.0254
pitch_m = pitch_in * 0.0254

rho = exp_data['rho']

time = exp_data['t']
fs_analog = exp_data['Fs_analog'].flatten()[0]


voltage = exp_data['data']
force_net = exp_data['force_net']
force_raw = exp_data['force']

voltage = voltage[:int(fs_analog * np.max(time)), :]
force_net = force_net[:int(fs_analog * np.max(time)), :]
force_raw = force_raw[:int(fs_analog * np.max(time)), :]

t_analog = np.linspace(0, len(force_raw)/fs_analog, len(force_raw)).flatten()

z_pulse = voltage[:len(t_analog), 6]
z_pulse = np.where(z_pulse > 1, 5, z_pulse)


rpm_sweep = exp_data['rpm_sweep']
U_inf = exp_data['U_inf']

In [40]:
indices = np.where(np.diff(z_pulse) > 1)[0]
rpm = 60/np.diff(t_analog.flatten()[indices])
t_rpm = t_analog.flatten()[indices[:-1]]

In [41]:
# Resample rpm to match fs_analog
num_samples = int(len(t_analog))
rpm_resampled = signal.resample(rpm, num_samples)

ValueError: invalid number of data points (0) specified

In [42]:
dt = 1/fs_analog
order = 2
cutoff = 10
nyq = 0.5 * fs_analog
normal_cutoff = cutoff / nyq

b, a = signal.butter(order, normal_cutoff)
z_l = z_d = signal.lfilter_zi(b, a)

Fx = signal.lfilter(b, a, force_net[:, 0], zi=z_l)[0]
Fy = signal.lfilter(b, a, force_net[:, 1], zi=z_l)[0]
Fz = signal.lfilter(b, a, force_net[:, 2], zi=z_l)[0]
Tx = signal.lfilter(b, a, force_net[:, 3], zi=z_l)[0]
Ty = signal.lfilter(b, a, force_net[:, 4], zi=z_l)[0]
Tz = signal.lfilter(b, a, force_net[:, 5], zi=z_l)[0]
rpm_filt = signal.lfilter(b, a, rpm_resampled, zi=z_l)[0]

In [43]:
n_filt = rpm_filt / 60


J = U_inf / (n_filt * diameter_m)
CT = (Fz / (rho * n_filt**2 * diameter_m**4)).T
CQ = (Tz / (rho * n_filt**2 * diameter_m**5)).T
CP = CQ * 2 * np.pi

ValueError: operands could not be broadcast together with shapes (8000234,) (1,2399000) 

In [44]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].plot(t_analog, Fx, "-")
axes[0, 0].set_xlabel("Time [s]")
axes[0, 0].set_ylabel("$F_x$ [N]")
axes[0, 0].grid()

axes[0, 1].plot(t_analog, Fy, "-")
axes[0, 1].set_xlabel("Time [s]")
axes[0, 1].set_ylabel("$F_y$ [N]")
axes[0, 1].grid()

axes[0, 2].plot(t_analog, Fz, "-")
axes[0, 2].set_xlabel("Time [s]")
axes[0, 2].set_ylabel("$F_z$ [N]")
axes[0, 2].grid()

axes[1, 0].plot(t_analog, Tx, "-")
axes[1, 0].set_xlabel("Time [s]")
axes[1, 0].set_ylabel("$T_x$ [N]")
axes[1, 0].grid()

axes[1, 1].plot(t_analog, Tx, "-")
axes[1, 1].set_xlabel("Time [s]")
axes[1, 1].set_ylabel("$T_y$ [N]")
axes[1, 1].grid()

axes[1, 2].plot(t_analog, Tx, "-")
axes[1, 2].set_xlabel("Time [s]")
axes[1, 2].set_ylabel("$T_z$ [N]")
axes[1, 2].grid()

plt.tight_layout()
plt.show()

In [45]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# First subplot
ax1.plot(t_analog, Fz, "-", label='Thrust', linewidth = 2, color='k')
ax1.set_ylim(-20, 20)

ax1_twiny = ax1.twinx()
ax1_twiny.plot(t_analog, rpm_filt, "-", label='RPM', linewidth=2, color='blue')
ax1_twiny.set_ylabel("RPM")
ax1_twiny.set_ylim(0, 10000)

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax1_twiny.get_legend_handles_labels()
ax1_twiny.legend(lines + lines2, labels + labels2, loc='upper left')

ax1.set_xlabel("Time [s]")
ax1.set_ylabel("T [N]")
ax1.grid()

# Second subplot
ax2.plot(t_analog, Tx, "-", label='Torque', linewidth = 2, color='red')
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Q [N.m]")
ax2.legend()
ax2.grid()

plt.tight_layout()
plt.show()

ValueError: x and y must have same first dimension, but have shapes (8000234,) and (2399000,)

In [46]:
# Perform PSD on Fz
f, Pxx_den = signal.welch(force_net[:, 2], fs_analog, nperseg=1024)

# Plot the PSD
plt.figure(figsize=(10, 6))
plt.semilogy(f, Pxx_den)
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.title('Power Spectral Density of Fz')
plt.grid()
plt.show()

In [47]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

title = f"APC {diameter_in[0][0]}x{pitch_in[0][0]}"

fig.suptitle(title, fontsize=16)
# First subplot
ax1.plot(t_analog, CT, "-", label='$C_T$', linewidth = 2, color='k')
ax1.set_ylim(0, 0.5)

# ax1_twiny = ax1.twinx()
# ax1_twiny.plot(t_analog, rpm_filt, "-", label='RPM', linewidth=2, color='blue')
# ax1_twiny.set_ylabel("RPM")
# ax1_twiny.set_ylim(0, 10000)

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax1_twiny.get_legend_handles_labels()
ax1_twiny.legend(lines + lines2, labels + labels2, loc='upper left')

ax1.set_xlabel("Time [s]")
ax1.set_ylabel("$C_T$")
ax1.grid()

# Second subplot
ax2.plot(t_analog, CP, "-", label='$C_P$', linewidth = 2, color='red')
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("$C_P$")
ax2.legend()
ax2.grid()

plt.tight_layout()
plt.show()

ValueError: x and y must have same first dimension, but have shapes (8000234,) and (2399000, 1)

In [ ]:
# Calculate the mean of Fz
start_index = int(5 / dt)
stop_index = int(350 / dt)

thrust_raw = force_net[start_index:stop_index, 2]
mean_Fz = np.cumsum(thrust_raw) / np.arange(1, len(thrust_raw) + 1)
std_Fz = np.sqrt(np.cumsum((thrust_raw - mean_Fz)**2) / np.arange(1, len(thrust_raw) + 1))

# Normalize the values with respect to the last calculated value of both mean and std
# mean_Fz /= mean_Fz[-1]
# std_Fz /= std_Fz[-1]

# mean_Fz -= 1
# std_Fz -= 1

# mean_Fz *= 100
# std_Fz *= 100

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

ax1.plot(t_analog[start_index:stop_index], mean_Fz, label='Normalized $\mu$ T %', color = "k")
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Normalized $\mu$ T %')
# ax1.set_ylim(-10, 10)
ax1.grid()
ax1.legend()

ax2.plot(t_analog[start_index:stop_index], std_Fz, label='Normalized $\sigma$ T %', color = "k")
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Normalized $\sigma$ T %')
ax2.grid()
# ax2.set_ylim(-10, 10)
ax2.legend()

plt.tight_layout()
plt.show()

<>:21: SyntaxWarning: invalid escape sequence '\m'
<>:23: SyntaxWarning: invalid escape sequence '\m'
<>:28: SyntaxWarning: invalid escape sequence '\s'
<>:30: SyntaxWarning: invalid escape sequence '\s'
<>:21: SyntaxWarning: invalid escape sequence '\m'
<>:23: SyntaxWarning: invalid escape sequence '\m'
<>:28: SyntaxWarning: invalid escape sequence '\s'
<>:30: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Lab7\AppData\Local\Temp\ipykernel_6080\2279483581.py:21: SyntaxWarning: invalid escape sequence '\m'
  ax1.plot(t_analog[start_index:stop_index], mean_Fz, label='Normalized $\mu$ T %', color = "k")
C:\Users\Lab7\AppData\Local\Temp\ipykernel_6080\2279483581.py:23: SyntaxWarning: invalid escape sequence '\m'
  ax1.set_ylabel('Normalized $\mu$ T %')
C:\Users\Lab7\AppData\Local\Temp\ipykernel_6080\2279483581.py:28: SyntaxWarning: invalid escape sequence '\s'
  ax2.plot(t_analog[start_index:stop_index], std_Fz, label='Normalized $\sigma$ T %', color = "k")
C:\Users\Lab7\AppData\Loc

In [36]:
# Calculate the mean of Tz
start_index = int(5 / dt)
stop_index = int(100 / dt)

torque_raw = force_net[start_index:stop_index, 5]
mean_Tz = np.cumsum(torque_raw) / np.arange(1, len(torque_raw) + 1)
std_Tz = np.sqrt(np.cumsum((torque_raw - mean_Tz)**2) / np.arange(1, len(torque_raw) + 1))

# Normalize the values with respect to the last calculated value of both mean and std
mean_Tz /= mean_Tz[-1]
std_Tz /= std_Tz[-1]

mean_Tz -= 1
std_Tz -= 1

mean_Tz *= 100
std_Tz *= 100

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

ax1.plot(t_analog[start_index:stop_index], mean_Tz, label='Normalized $\mu$ Q %', color = "k")
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Normalized $\mu$ Q %')
ax1.set_ylim(-10, 10)
ax1.grid()
ax1.legend()

ax2.plot(t_analog[start_index:stop_index], std_Tz, label='Normalized $\sigma$ Q %', color = "k")
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Normalized $\sigma$ Q %')
ax2.grid()
ax2.set_ylim(-10, 10)
ax2.legend()

plt.tight_layout()
plt.show()

<>:21: SyntaxWarning: invalid escape sequence '\m'
<>:23: SyntaxWarning: invalid escape sequence '\m'
<>:28: SyntaxWarning: invalid escape sequence '\s'
<>:30: SyntaxWarning: invalid escape sequence '\s'
<>:21: SyntaxWarning: invalid escape sequence '\m'
<>:23: SyntaxWarning: invalid escape sequence '\m'
<>:28: SyntaxWarning: invalid escape sequence '\s'
<>:30: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Lab7\AppData\Local\Temp\ipykernel_6080\1996771505.py:21: SyntaxWarning: invalid escape sequence '\m'
  ax1.plot(t_analog[start_index:stop_index], mean_Tz, label='Normalized $\mu$ Q %', color = "k")
C:\Users\Lab7\AppData\Local\Temp\ipykernel_6080\1996771505.py:23: SyntaxWarning: invalid escape sequence '\m'
  ax1.set_ylabel('Normalized $\mu$ Q %')
C:\Users\Lab7\AppData\Local\Temp\ipykernel_6080\1996771505.py:28: SyntaxWarning: invalid escape sequence '\s'
  ax2.plot(t_analog[start_index:stop_index], std_Tz, label='Normalized $\sigma$ Q %', color = "k")
C:\Users\Lab7\AppData\Loc